In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection

In [2]:
V, F = [], []
with open("brocco.obj") as f:
   for line in f.readlines():
       if line.startswith('#'):
           continue
       values = line.split()
       if not values:
           continue
       if values[0] == 'v':
           V.append([float(x) for x in values[1:4]])
       elif values[0] == 'f':
           F.append([int(x.split('/')[0]) for x in values[1:4]])
V, F = np.array(V), np.array(F)-1

In [3]:
V = (V-(V.max(0)+V.min(0))/2)/max(V.max(0)-V.min(0))

In [4]:
T = V[F][...,:2]

In [5]:
def show_brocco():
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_axes([0,0,1,1], xlim=[-1,+1], ylim=[-1,+1],
                      aspect=1, frameon=False)
    collection = PolyCollection(T, closed=True, linewidth=0.1,
                                facecolor="None", edgecolor="black")
    ax.set_xticks([])
    ax.set_yticks([])
    ax.add_collection(collection)
    plt.show()

In [6]:
def frustum(left, right, bottom, top, znear, zfar):
    M = np.zeros((4, 4), dtype=np.float32)
    M[0, 0] = +2.0 * znear / (right - left)
    M[1, 1] = +2.0 * znear / (top - bottom)
    M[2, 2] = -(zfar + znear) / (zfar - znear)
    M[0, 2] = (right + left) / (right - left)
    M[2, 1] = (top + bottom) / (top - bottom)
    M[2, 3] = -2.0 * znear * zfar / (zfar - znear)
    M[3, 2] = -1.0
    return M

def perspective(fovy, aspect, znear, zfar):
    h = np.tan(0.5*np.radians(fovy)) * znear
    w = h * aspect
    return frustum(-w, w, -h, h, znear, zfar)

In [7]:
#V = V - (0,0,3.5)
#V = np.c_[V, np.ones(len(V))] @ perspective(25,1,1,100).T
#V /= V[:,3].reshape(-1,1)

In [8]:
def translate(x, y, z):
    return np.array([[1, 0, 0, x],
                     [0, 1, 0, y],
                     [0, 0, 1, z],
                     [0, 0, 0, 1]], dtype=float)

def xrotate(theta):
    t = np.pi * theta / 180
    c, s = np.cos(t), np.sin(t)
    return np.array([[1, 0,  0, 0],
                     [0, c, -s, 0],
                     [0, s,  c, 0],
                     [0, 0,  0, 1]], dtype=float)

def yrotate(theta):
    t = np.pi * theta / 180
    c, s = np.cos(t), np.sin(t)
    return  np.array([[ c, 0, s, 0],
                      [ 0, 1, 0, 0],
                      [-s, 0, c, 0],
                      [ 0, 0, 0, 1]], dtype=float)

In [9]:
model = xrotate(20) @ yrotate(45)
view  = translate(0,0,-3.5)
proj  = perspective(25, 1, 1, 100)
MVP   = proj  @ view  @ model

In [10]:
T =  V[:,:,:2]
Z = -V[:,:,2].mean(axis=1)
I = np.argsort(Z)
T = T[I,:]

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [11]:
V

array([[-0.20237535,  0.4070471 , -0.15220555],
       [-0.12185466,  0.39247074, -0.21505584],
       [-0.19770595,  0.31051999, -0.31994937],
       ...,
       [ 0.06619184,  0.42049118, -0.23268727],
       [ 0.05850207,  0.4265306 , -0.1740238 ],
       [ 0.06567441,  0.42707767, -0.17809543]])